In [1]:
from langchain_playground.tools.youtube.summarizer import scrape_youtube

scrape_response = scrape_youtube(youtube_url="https://youtu.be/2HvLcJDdyWY")
scrape_response.parsed_transcript

'Folks, a once-in-a-lifetime event is \n coming in 2026, and you need to consider \n doing very specific things by December \n 31st. Now, the setup for 2026 is \n incredibly unusual, incredibly unique, \n and incredibly opportune for those who \n are paying attention. We\'ve got fiscal \n policy, monetary policy, and \n deregulation working together in a way \n that we really have never seen before. \n at the same time where you have this \n massive new technology AI that has taken \n the world by storm and is set to draw in \n trillions upon trillions of dollars \n worth of capex. This video will break \n down as follows. How the market is \n setting up for 2026. The specific data \n that points to what\'s going on. Number \n two, the top stocks that are set to \n succeed in 2026. And of course, number \n three, the checklist of seven things you \n must do by December 31st. And then it\'ll \n be time for our sponsored segment on \n Quantum Bioarma, ticker symbol Q&M on \n the NASDAQ. 

In [2]:
print(scrape_response.parsed_transcript)

Folks, a once-in-a-lifetime event is 
 coming in 2026, and you need to consider 
 doing very specific things by December 
 31st. Now, the setup for 2026 is 
 incredibly unusual, incredibly unique, 
 and incredibly opportune for those who 
 are paying attention. We've got fiscal 
 policy, monetary policy, and 
 deregulation working together in a way 
 that we really have never seen before. 
 at the same time where you have this 
 massive new technology AI that has taken 
 the world by storm and is set to draw in 
 trillions upon trillions of dollars 
 worth of capex. This video will break 
 down as follows. How the market is 
 setting up for 2026. The specific data 
 that points to what's going on. Number 
 two, the top stocks that are set to 
 succeed in 2026. And of course, number 
 three, the checklist of seven things you 
 must do by December 31st. And then it'll 
 be time for our sponsored segment on 
 Quantum Bioarma, ticker symbol Q&M on 
 the NASDAQ. a company with two potential

In [2]:
from langchain_playground.llm import ChatOpenRouter

llm = ChatOpenRouter(
    model="x-ai/grok-4.1-fast",
    reasoning_effort="low",
    web_search=True,
    web_search_max_results=3,
)
llm.invoke("What is the price of NVDA")

2025-12-12 21:38:36,849 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="**As of December 12, 2025 (pre-market, around 8-9 AM EST):**\n\n- **Pre-market price**: ~$182.18–$182.66 (up ~0.7–1.0% from close)\n  - Yahoo Finance: $182.18 (+0.69%)\n  - CNBC: $182.66 (+0.96% after hours/pre-market)\n\n**Latest regular session close (Dec 11, 4:00 PM EST)**: **$180.93** (down 1.55% from prev close of $183.78)\n\n**Key stats**:\n- Day's range (Dec 11): $176.62–$181.32\n- Volume: ~165–177 million shares\n- Market cap: ~$4.40 trillion\n- 52-week range: $86.62–$212.19\n\nPrices fluctuate in real-time; check Yahoo Finance or CNBC for live updates. Recent pressure from AI sector concerns and chip earnings (e.g., Broadcom).", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1803, 'prompt_tokens': 13327, 'total_tokens': 15130, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 1253, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_deta

In [ ]:
from rich import print as rprint

from langchain_playground.tools.youtube import summarize_video

response = summarize_video("https://www.youtube.com/watch?v=OOIjHeeRWJw")
rprint(response)

2025-12-12 20:48:09,619 - WARNING - USER_AGENT environment variable not set, consider setting it to identify your requests.
2025-12-12 20:48:17,961 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-12 20:48:36,739 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


✅ Complete: quality 100%
🎯 Final: quality 100%, 1 iterations


Analysis(
    title='Oracle Earnings Deep Dive: AI Capex Surge and Market Impact',
    summary="The video analyzes Oracle's recent Q2 earnings, where the stock fell 13% despite strong remaining 
performance obligations (RPO) growth to $523 billion, driven by AI demand from clients like Meta and Nvidia. Key 
highlights include a massive capex increase to $50 billion for FY2026 to build AI infrastructure, leading to 
negative free cash flow of $10 billion in the quarter. While short-term revenue recognition remains limited, with 
only $4 billion added to FY2027 guidance, the speaker views this as bullish for AI chipmakers like Nvidia and AMD, 
dismissing bearish sentiment as a buying opportunity for the AI ecosystem.",
    takeaways=[
        "Oracle's RPO backlog exploded by $68 billion QoQ to $523 billion, signaling robust AI demand but delayed 
revenue recognition.",
        'Capex guidance surged 43% to $50 billion for FY2026, fueling AI infrastructure for hyperscale growth and 
benefiting suppliers like Nvidia.',
        'Negative free cash flow from heavy capex is temporary; revenue acceleration expected as capacity comes 
online post-2027.',
        'Oracle adopts chip neutrality, selling stakes in custom CPU ventures to focus on Nvidia, AMD, and Intel 
GPUs.',
        'Despite debt concerns and market fears, the dip presents a buying opportunity for AI stocks including 
Oracle, Nvidia, and AMD.'
    ],
    chapters=[
        Chapter(
            header='Market Reaction to Oracle Earnings',
            summary="Oracle's stock dropped 13% after earnings, pulling down AI-related stocks like Nvidia (-3.3%),
AMD (-3.6%), Credo Technology (-4.5%), and Neocloud players (-5.75%). Analysts focused on short-term AI cloud 
monetization, capex, and cash flow amid expected massive AI investments.",
            key_points=[
                'Earnings triggered broad AI market sell-off despite anticipated short-term volatility.',
                'RPO increased dramatically but most additions deferred beyond FY2026.',
                'Investor concerns centered on capex burning free cash flow without immediate revenue.'
            ]
        ),
        Chapter(
            header='RPO Backlog Explosion',
            summary="Oracle's RPO reached $523 billion, up $68 billion QoQ from major contracts with Meta, Nvidia, 
and others. Only $4 billion added to FY2027 revenue guidance (from $34B to $38B), with growth accelerating later 
due to capacity constraints.",
            key_points=[
                'RPO growth of 68B QoQ highlights insatiable AI demand.',
                'Short-term additions limited; FY2027 sees 10-15% uplift over prior guidance.',
                'Deferred recognition fuels bearish optics but validates long-term AI thesis.'
            ]
        ),
        Chapter(
            header='Capex Surge and Cash Flow Strain',
            summary='Capex guidance exploded to $50B for FY2026, up $15B (43%) from prior $35B estimate, rivaling 
hyperscalers. Q2 free cash flow was -$10B (operating cash +$2.1B overwhelmed by $12B capex), third consecutive 
negative quarter.',
            key_points=[
                'Capex ramp bullish for Nvidia/AMD as Oracle buys chips aggressively now.',
                'Front-loading investments to enable faster revenue once infrastructure online.',
                'Lumpiness from GPU deployment lag creates temporary cash burn fear.'
            ]
        ),
        Chapter(
            header='Risks: Debt, Overbuilding, and Execution',
            summary='High debt-to-equity ratio, low credit rating, and CDS spreads over 125bps raise concerns. 
Oracle risks overbuilding if AI demand falters. Chip neutrality shift: sold stake in Marvell CPU venture to 
prioritize best GPUs from Nvidia/AMD/Intel.',
            key_points=[
                'Debt balloons with capex; potential credit risks if growth disappoints.',
                'Overbuilding analogy: filling tank for uncertain long trip.',
    

In [ ]:
from langchain.tools import tool

from langchain_playground.tools import youtube_loader


@tool
def youtube_loader_tool(youtube_url: str) -> str:
    """Scrape a YouTube video and return its transcript and metadata."""
    return youtube_loader(youtube_url)


tools = [youtube_loader_tool]

In [ ]:
from langchain.agents import create_agent

from langchain_playground import ChatOpenRouter

# Create a model for the agent (without structured output, as agents handle tool calling)
agent_llm = ChatOpenRouter(
    model="openai/gpt-5-mini",
    temperature=0,
    reasoning_effort="medium",
)

# Create a simple agent with the YouTube scraper tool
youtube_agent = create_agent(
    model=agent_llm,
    tools=tools,
)